# Анализ курса валют

## 1. Введение

Этот Jupyter Notebook демонстрирует получение, обработку и визуализацию данных о курсе доллара США к российскому рублю, используя данные Центрального банка Российской Федерации.

## 2. Импорт библиотек

In [2]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from ipywidgets import interact, DatePicker
from datetime import datetime, timedelta

## 3. Константы и настройки

In [3]:
# URL для запроса данных с ЦБ РФ (шаблон)
CURRENCY_URL_TEMPLATE = "https://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={start_date}&date_req2={end_date}&VAL_NM_RQ=R01235"

# Форматы даты
DATE_FORMAT = "%d.%m.%Y"
DATE_FORMAT_REQUEST = "%d/%m/%Y"

# Период для загрузки данных (по умолчанию - 1 год)
DEFAULT_PERIOD = timedelta(days=365)

## 4. Функция для получения данных

In [4]:
def fetch_currency_data(start_date, end_date):
    """
    Получает данные о курсе валюты с сайта ЦБ РФ.

    Args:
        start_date (str): Дата начала в формате DD/MM/YYYY.
        end_date (str): Дата окончания в формате DD/MM/YYYY.

    Returns:
        pandas.DataFrame: DataFrame с данными о курсе валюты, 
                           или пустой DataFrame, если данные не получены.
    """
    url = CURRENCY_URL_TEMPLATE.format(start_date=start_date, end_date=end_date)
    try:
        response = requests.get(url)
        response.raise_for_status()  # Проверка на ошибки HTTP

        tree = ET.ElementTree(ET.fromstring(response.content))
        root = tree.getroot()

        data = []
        for record in root.findall('Record'):
            date_str = record.attrib['Date']
            value_str = record.find('Value').text.replace(',', '.')
            try:
                date = datetime.strptime(date_str, DATE_FORMAT)
                value = float(value_str)
                data.append([date, value])
            except (ValueError, TypeError):
                print(f"Ошибка при обработке записи: Дата - {date_str}, Значение - {value_str}")
                continue

        df = pd.DataFrame(data, columns=['Date', 'USD/RUB'])
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        return df
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе данных: {e}")
        return pd.DataFrame()

## 5. Функция для построения графика

In [5]:
def plot_currency(start_date, end_date):
    """Строит график курса валюты."""
    plt.figure(figsize=(10, 6))  # Размер графика
    
    df_filtered = df.loc[start_date:end_date]
    plt.plot(df_filtered.index, df_filtered['USD/RUB'], marker='o', linestyle='-', color='blue')

    plt.title('Курс USD к RUB по данным ЦБ РФ')
    plt.xlabel('Дата')
    plt.ylabel('Курс')
    plt.grid(True)
    plt.xticks(rotation=45, ha='right') # Поворот меток на оси X
    plt.tight_layout() #  Для предотвращения обрезания меток
    plt.show()

## 6. Загрузка и отображение данных

In [6]:
# Дата окончания - текущая дата
end_date_initial = datetime.now()
# Дата начала - год назад
start_date_initial = end_date_initial - DEFAULT_PERIOD

# Преобразование дат в строки для запроса
start_date_str = start_date_initial.strftime(DATE_FORMAT_REQUEST)
end_date_str = end_date_initial.strftime(DATE_FORMAT_REQUEST)

# Загрузка данных
df = fetch_currency_data(start_date_str, end_date_str)



## 7. Описание данных

DataFrame `df` содержит данные о курсе USD/RUB за период с {start_date} по {end_date}.  Всего в DataFrame {len(df)} записей.  Значения в столбце 'USD/RUB' представляют собой официальный курс доллара США к российскому рублю, установленный Центральным банком РФ.

In [7]:
# Проверка на наличие данных
if not df.empty:
    # Виджеты для выбора дат
    date_start_picker = DatePicker(description='Дата начала', value=start_date_initial.date())
    date_end_picker = DatePicker(description='Дата конца', value=end_date_initial.date())

    # Интерактивный график
    interact(plot_currency, start_date=date_start_picker, end_date=date_end_picker);
else:
    print("Не удалось загрузить данные для построения графика.")

interactive(children=(DatePicker(value=datetime.date(2024, 1, 9), description='Дата начала', step=1), DatePick…